# CS 5661 Homework 1
# Kevin Tran

### a. Read the dataset file “Cancer.csv” (you should download it from CSNS), and store it in a Pandas DataFrame. Check out the dataset. As you see, the dataset includes 9 numerical features. The last column is the binary label (“1” means it is a malignant cancer, “0” means it is a benign tumor). You will use all 9 features in this homework.


In [1]:
import numpy as np
import pandas as pd

cancer_df = pd.read_csv("Cancer_small.csv")

cancer_df.head()

,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Malignant_Cancer
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0


### b. Use sklearn functions (see class tutorials for details) to split the dataset into testing and training sets with the following parameters: test_size=0.3, random_state=2.


In [2]:
from sklearn.model_selection import train_test_split

features = list(cancer_df.loc[:, cancer_df.columns != 'Malignant_Cancer'])
label = ['Malignant_Cancer']

x = cancer_df[features]
y = cancer_df[label]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=2)

print(x_train.head())
print(y_test.head())

     Clump_Thickness  Uniformity_of_Cell_Size  Uniformity_of_Cell_Shape  \
134                5                        8                         4   
28                 4                        1                         1   
65                 5                       10                         6   
0                  5                        1                         1   
27                 1                        1                         1   

     Marginal_Adhesion  Single_Epithelial_Cell_Size  Bare_Nuclei  \
134                 10                            5            8   
28                   3                            2            1   
65                   1                           10            4   
0                    1                            2            1   
27                   1                            2            1   

     Bland_Chromatin  Normal_Nucleoli  Mitoses  
134                9               10        1  
28                 3                1     

### c. Use “Decision Tree Classifier” to predict Cancer based on the training/testing datasets that you built in part (b). Then, calculate and report the accuracy of your classifier. Use this command to define your tree: my_DecisionTree = DecisionTreeClassifier(random_state=2).


In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

my_decisiontree = DecisionTreeClassifier(random_state=2)
my_decisiontree.fit(x_train, y_train)
y_predict_dt = my_decisiontree.predict(x_test)

dec_tree_accuracy = accuracy_score(y_test, y_predict_dt)
print(dec_tree_accuracy)

0.8666666666666667


### d.  Now, we want to perform “Bagging” based on 19 “base decision tree classifiers”. Note: you should write your own code to perform Bagging (don’t use scikit-learn functions for Bagging!) To do so, you need to perform bootstrapping first. You can write a “for” loop with loop variable i=0...18. In each iteration of the loop, you have to:


#### - make a bootstrap sample of the original “Training” Dataset (build in part(b)) with the size of bootstarp_size = 0.8*(Size of the original dataset). You can use the following command to generate a random bootstrap dataset (“i" is the variable of the loop, so the random_state changes in each iteration): 
### resample(X_train, n_samples = bootstarp_size , random_state=i , replace = True)


#### -  Define and train a new base decision tree classifier on this dataset in each iteration:
### Base_DecisionTree = DecisionTreeClassifier(random_state=2).


#### -  Test “this base classifier” on the original “Testing” Dataset build in part(b), and save the prediction results for all testing samples.


In [4]:
from sklearn.utils import resample


n_iterations = 19
bootstrap_size = int(len(cancer_df) * .8)

bootstrap = list()

for i in range(n_iterations):
    train = resample(x_train, n_samples=bootstrap_size , random_state=i , replace=True)
    test = y_train.loc[train.index.values]

    Base_DecisionTree = DecisionTreeClassifier(random_state=2)
    Base_DecisionTree.fit(train, test)
    
    y_predict_dtc = Base_DecisionTree.predict(x_test)
    boot_score = accuracy_score(y_test, y_predict_dtc) 
    bootstrap.append(boot_score)

print('Original Decision Tree Accuracy: {}'.format(dec_tree_accuracy))
print()
print('Sample Values: {}'.format(bootstrap))

Original Decision Tree Accuracy: 0.8666666666666667

Sample Values: [0.7777777777777778, 0.8666666666666667, 0.8666666666666667, 0.8444444444444444, 0.8222222222222222, 0.8444444444444444, 0.8444444444444444, 0.8444444444444444, 0.8888888888888888, 0.8, 0.8444444444444444, 0.8444444444444444, 0.8444444444444444, 0.8888888888888888, 0.8222222222222222, 0.8444444444444444, 0.8444444444444444, 0.8444444444444444, 0.8666666666666667]


#### - Perform Voting to make the final decision on each data sample based on the votes of all 19 classifiers. Finally, calculate and report the accuracy of your Bagging method.


In [5]:
from sklearn.ensemble import VotingClassifier

clf1 = DecisionTreeClassifier(random_state=2)

my_vote = VotingClassifier(estimators=[('lr', clf1)])
my_vote = my_vote.fit(x_train, y_train)
y_predict_vote = my_vote.predict(x_test)

vote_accuracy = accuracy_score(y_test, y_predict_vote)
print(vote_accuracy)


0.8666666666666667


/home/niivek/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/niivek/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/niivek/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Finally, calculate and report the accuracy of your Bagging method.

### e.  Use scikit-learn “Adaboost” classifier to predict Cancer based on the training/testing datasets that you built in part (b). Then, calculate and report the accuracy of your classifier. Use this command to import and define your classifier: from sklearn.ensemble import AdaBoostClassifier my_AdaBoost = AdaBoostClassifier(n_estimators = 19,random_state=2)


In [6]:
from sklearn.ensemble import AdaBoostClassifier

my_AdaBoost = AdaBoostClassifier(n_estimators = 19, random_state=2)
my_AdaBoost.fit(x_train, y_train)
y_predict_ada = my_AdaBoost.predict(x_test)

ada_boost_accuracy = accuracy_score(y_test, y_predict_ada)
print(ada_boost_accuracy)

0.9333333333333333


/home/niivek/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### f. Use scikit-learn “Random Forest” classifier to predict Cancer based on the training/testing datasets that you built in part (b). Then, calculate and report the accuracy of your classifier. Use this command to import and define your classifier: from sklearn.ensemble import RandomForestClassifier my_RandomForest = RandomForestClassifier(n_estimators = 19, bootstrap = True, random_state=2)


In [7]:
from sklearn.ensemble import RandomForestClassifier

my_RandomForest = RandomForestClassifier(n_estimators=19, bootstrap=True, random_state=2)
my_RandomForest.fit(x_train, y_train)
y_predict_rf = my_RandomForest.predict(x_test)
random_forest_accuracy = accuracy_score(y_test, y_predict_rf)
print(random_forest_accuracy)

0.9555555555555556


/home/niivek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
